### SqueezeNet 

### SqueezeNet: AlexNet-level accuracy with 50x fewer parameters and< 0.5 MB model size  (Iandola F. N. et al., DeepScale & UC Berkeley & Stanford, 2016)

*SqueezeNet achieves
AlexNet-level accuracy on ImageNet with 50x fewer parameters. Additionally,
with model compression techniques, we are able to compress SqueezeNet to less
than 0.5MB (510× smaller than AlexNet)*


[Paper](https://arxiv.org/abs/1602.07360)

In [1]:
import os
import numpy as np
import netron
import torch
import torch.nn as nn

import plotly.offline as offline
offline.init_notebook_mode(connected=True)

assert torch.cuda.is_available() is True
%load_ext watermark

In [2]:
%watermark -p torch,ignite,numpy,netron,sklearn,pandas,plotly

torch  : 1.10.2
ignite : 0.4.8
numpy  : 1.22.1
netron : 5.5.5
sklearn: 0.24.2
pandas : 1.4.1
plotly : 5.6.0



#### SqueezeNet

Motivation:

* More efficient distributed training;

* Less overhead when exporting new models to clients;

* FPGA [1](https://habr.com/ru/post/505838/), [2](https://web.archive.org/web/20070412183416/http://filebox.vt.edu/users/tmagin/history.htm) usability with max 10Mb on-chip memory.

Main ideas:

* Replace 3x3 filters with 1x1 filters;
* Decrease the number of input channels to 3x3 filters with "squeeze" layers;
* Downsample late in the network so that convolution layers have large activation maps. Intuition: large activation maps can lead to higher classification accuracy;
* General building block - fire module.

#### Fire module

In [3]:
class Fire(nn.Module):
    def __init__(self, inplanes: int, squeeze_planes: int, expand1x1_planes: int, expand3x3_planes: int) -> None:
        super().__init__()
        self.inplanes = inplanes
        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.ReLU(inplace=True)
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes, kernel_size=1)
        self.expand1x1_activation = nn.ReLU(inplace=True)
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes, kernel_size=3, padding=1)
        self.expand3x3_activation = nn.ReLU(inplace=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.squeeze_activation(self.squeeze(x))
        return torch.cat(
            [self.expand1x1_activation(self.expand1x1(x)), self.expand3x3_activation(self.expand3x3(x))], 1
        )

In [4]:
fire_module = Fire(96, 16, 64, 64)
x = torch.Tensor(np.random.normal(size=(1, 96, 64, 64)))
model_path = os.path.join('onnx_graphs', 'fire_module.onnx')
torch.onnx.export(fire_module, x, model_path,
                  input_names=['input'], output_names=['output'], opset_version=10)
netron.start(model_path, 30000)

Serving 'onnx_graphs/fire_module.onnx' at http://localhost:30000


('localhost', 30000)

<img src="../assets/1_squeezenet.png" width="750">

<img src="../assets/2_squeezenet.png" width="750">

#### Torch [implementation](https://github.com/pytorch/vision/blob/50608fbc263da386ad7abf7b32bd32ed7f691170/torchvision/models/squeezenet.py#L36)

#### FPGA [implementation](https://github.com/dgschwend/zynqnet)

*FPGA, field-programmable gate array

#### Your training code here

In [ ]:
# Define data transformation pipeline.


# Initialize dataset and dataloaders.


# Initialize pretrained network, replace Linear layer with a new one for your dataset.


# Initialize optimizer, loss function and training procedure with handlers/callbacks.

#### References

* https://github.com/forresti/SqueezeNet
* https://onnx.ai/
* https://pytorch.org/docs/stable/index.html
* https://lankas.github.io/15-618Project/
* https://github.com/dgschwend/zynqnet